In [2]:
pip install dash sqlalchemy pymysql pandas plotly

Note: you may need to restart the kernel to use updated packages.


In [1]:
from sqlalchemy import create_engine
import pandas as pd
#Db connectivity
user = 'root'
password = 'Naresh'
host = 'localhost'
database = 'airport_db'
#craete connection engine
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')
# write query data
query = "Select * from airports2"
#create dataframe
df = pd.read_sql(query,engine)
df.head()


,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254,2008-10-01,122049,86219,39.140999,-96.670799,NA,NA
1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103,1990-11-01,284093,76034,44.124599,-123.211998,44.2541008,-121.1500015
2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103,1990-12-01,284093,76034,44.124599,-123.211998,44.2541008,-121.1500015
3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103,1990-10-01,284093,76034,44.124599,-123.211998,44.2541008,-121.1500015
4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156,1990-02-01,147300,76034,42.374199,-122.873001,44.2541008,-121.1500015


In [2]:
#create simple dash application
from dash import Dash, dcc, html
import plotly.express as px

#create dash app variable
app = Dash(__name__)


#plot using plotly (passengers data over distance)
fig = px.scatter(df, x='Distance', y='Passengers', color='Origin_airport', title='Passenger vs distance', labels={'Passengers': 'Passengers', 'Distance':'Distance(Miles)'})


app.layout = html.Div(children = [
    html.H1(children='Flight Dashboard'),
    dcc.Graph(
        id='passenger-distance graph',
        figure= fig
    ),
    html.Div(children='Dashboard for visualization of flight data')
]   
)

if __name__ == '__main__':
    app.run(debug=True, port=8051)


In [3]:
#create simple dash application
from dash import Dash, dcc, html
import plotly.express as px

#create dash app variable
app = Dash(__name__)


#plot using plotly (passengers data over distance)
fig = px.scatter(df, x='Distance', y='Passengers', color='Origin_airport', title='Passenger vs distance', labels={'Passengers': 'Passengers', 'Distance':'Distance(Miles)'})
# adding barchart also
bar_fig = px.bar(df, x='Origin_airport', y='Seats', color='Destination_airport', title='Seats per flight', labels={'Seats': 'No of seats', 'Origin_airport':'Origin_airport'})


app.layout = html.Div(children = [
    html.H1(children='Flight Dashboard'),
    dcc.Graph(
        id='passenger-distance graph',
        figure= fig
    ),
    dcc.Graph(
        id='Seats bar chart',
        figure= bar_fig
    ),
    html.Div(children='Dashboard for visualization of flight data')
]   
)

if __name__ == '__main__':
    app.run(debug=True, port=8051)


In [ ]:
# Add filteration
from dash import dcc, html, Input, Output
import dash
import plotly.express as px
import pandas as pd
#create dash app
app = dash.Dash(__name__)
bar_fig = px.bar(df, x='Origin_airport', y='Seats', color='Destination_airport', title='Seats per flight', labels={'Seats': 'No of seats', 'Origin_airport':'Origin_airport'})
app.layout = html.Div([
    html.H1('Flight Dashboard'),
    dcc.Dropdown(
        id = 'Origin-airport-dropdown',
        options = [{'label': airport, 'value': airport}for airport in df['Origin_airport'].unique()],
        placeholder='Select an origin Airport',
        multi = True
    ),
    dcc.DatePickerRange(
        id = 'date-picker-range',
        start_date = df['Fly_date'].min(),
        end_date= df['Fly_date'].max(),
        display_format = 'YYYY-MM-DD'
    ),
    dcc.Graph(id = 'filtered-passenger-distance-graph'),
    dcc.Graph(id = 'filtered-seats-bar-chart' )
 ])
    ##callback for interactivity
@app.callback(
    [Output('filtered-passenger-distance-graph', 'figure'),
     Output('filtered-seats-bar-chart', 'figure')],
    [Input('Origin-airport-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range','end_date'),]
)
   
def update_graphs(selected_airports, start_date, end_date):
    # filter data based on selected airports, start date and end date
    filtered_df = df[(df['Fly_date']>= start_date) & (df['Fly_date']<= end_date)]
    if selected_airports:
        filtered_df = filtered_df['Origin_airport'].isin(selected_airports)
    #update Scatter plot
    scatter_fig = px.scatter(filtered_df, x='Distance', y='Passengers', color='Origin_airport',
                 title='Filtered Passenger vs distance', labels={'Passengers': 'Passengers', 'Distance':'Distance(Miles)'})
    bar_fig = px.bar(filtered_df, x='Origin_airport', y='Seats', color='Destination_airport', title='Filtered Seats per flight')
    return scatter_fig, bar_fig
    


if __name__ == '__main__':
    app.run(debug=True, port= 8052)